In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
pd.__version__


In [ ]:
spark.conf.set("fs.azure.account.key.stupramonitoreomercado.dfs.core.windows.net", "xxx")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
#spark.conf.set("spark.sql.shuffle.partitions",3000)
#spark.conf.set("spark.default.parallelism",3000)

In [ ]:
df = spark.read.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones_v2.parquet")
#display(from_json(df2.select("col"))
display(df)

In [ ]:
df = df.withColumn("id", F.monotonically_increasing_id())
display(df)

In [ ]:
df.count()

In [ ]:

df.filter(df.LLAVE_COMP == "27361184-129570126002015").display()

In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView("archivo_rangos")

# divipola donde fue la transaccion + cod oficina orip + matricula inmobiliaria + num anotacion (hasta 5 digitos) +  cod nat juridica  
df_key_transaccion = spark.sql("""
SELECT distinct  COD_DANE || '-' ||    lpad(split(MATRICULA, '-')[0], 3, '0')    || '-' || lpad( split(MATRICULA, '-')[1], 6, '0' )
|| '-' ||  lpad( NRO_ANOTACION, 5, '0') || '-' || COD_NATURALEZA  as pk, 
COD_DANE, lpad(ORIP, 3, '0')  as ORIP , 
lpad(split(MATRICULA, '-')[0], 3, '0')   AS MATRICULA_ORIP,  lpad( split(MATRICULA, '-')[1], 6, '0' ) AS MATRICULA_NUMERO, 
 lpad( NRO_ANOTACION, 5, '0') AS NRO_ANOTACION, COD_NATURALEZA, DEPARTAMENTO,  MUNICIPIO, VEREDA, TIPO_PREDIO, NATURALEZA_JURIDICA, DIRECCION, 
 SISTEMA
FROM archivo_rangos
""")
# No se pueden adicionar porque se duplican registros: NRO_PROPIETARIOS   , ANIO_RADICACION, ESTADO_FOLIO
df_key_transaccion.display()



In [ ]:
df_key_transaccion.count()
# 1631192

In [ ]:
df_key_transaccion.write.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones_principal.parquet")

In [ ]:

profile = ProfileReport(df_key_transaccion.select("*").toPandas() , title="Pandas Profiling Report - Transacciones inmobiliarias - Llave primaria")
#displayHTML(profile.to_html())
dbutils.fs.put("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones_principal.parquet_profile.html", profile.to_html() )

In [ ]:
df_key_transaccion.printSchema()

In [ ]:
df_key_transaccion.createOrReplaceTempView("key_transaccion")
spark.sql("""
SELECT count(*)
FROM key_transaccion  where ORIP <> MATRICULA_ORIP
""").show()

In [ ]:
## TODO generar archivo con datos de las escrituras (valor, lindero)   NRO_PROPIETARIOS   , ANIO_RADICACION, ESTADO_FOLIO
df_key_linderos = spark.sql("""
SELECT distinct  COD_DANE || '-' ||    lpad(split(MATRICULA, '-')[0], 3, '0')    || '-' || lpad( split(MATRICULA, '-')[1], 6, '0' )
|| '-' ||  lpad( NRO_ANOTACION, 5, '0') || '-' || COD_NATURALEZA  as pk, 
ANIO_RADICACION
FROM archivo_rangos
""")  
df_key_linderos.count()

#  1631192  LINDERO
#  1631201  ANIO_RADICACION 
#  1631213  LINDERO, VALOR
#  1631218  NRO_PROPIETARIOS,  ANIO_RADICACION, ESTADO_FOLIO, , ANIO_RADICACION, 
#  1631218  LINDERO, VALOR, ANIO_RADICACION, ESTADO_FOLIO


In [ ]:
df_dir_common = df.withColumn('word', F.explode(F.split(F.col('LINDERO'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

df_dir_common.show()

In [ ]:
df_dir_common.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/palabras_comunes_linderos.json")

In [ ]:
df_sin_informacion = df.filter(df.TIPO_PREDIO == "SIN INFORMACION")
df_sin_informacion.count()

In [ ]:
display(df_sin_informacion)

In [ ]:
df_sin_informacion.createOrReplaceTempView("t_sin_info")
df_clean =   spark.sql("""
SELECT  *,
 trim( regexp_replace(  regexp_replace( upper(DIRECCION),  ' +', ' ') ,  ',+', ','    ) )as DIRECCION_CLEAN  
FROM t_sin_info
""")

In [ ]:
df_dir_common = df_clean.withColumn('word', F.explode(F.split(F.col('DIRECCION_CLEAN'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

df_dir_common.show()

In [ ]:
df_dir_common.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/sin_info_dir_palabras_comunes.json")

In [ ]:
df_clean2 = df_clean.filter(df_clean.DIRECCION_CLEAN.contains("CALLE") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("CARRERA") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("DIAGONAL") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("MANZANA") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("URBANIZACION") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("APARTAMENTO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("EDIFICIO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("BARRIO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("CONJUNTO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("HORIZONTAL") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("CEMENTERIO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("TRANSVERSAL") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("P.H.") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("LOCAL") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("APTO") == False)
df_clean2 = df_clean2.filter(df_clean2.DIRECCION_CLEAN.contains("PARQUEADERO") == False)


# 51038
# 46314  == "Rural inferido"
# número de intervinientes 
#  Divipola_Mun_0001  = "divipola_orip"
#  divipola donde fue la transaccion + cod oficina orip + matricula inmobiliaria + num anotacion (hasta 5 digitos) +  cod nat juridica  

# zre_nombre == nombre orip

# reporte: en cuales aparece área y por cada registro cuantas veces aparece la palabra área  en el campo lindero
#  si aparece 1: esa es el área
#  si aparece > de 1 :   (generar reporte para evaluar el patrón de texto ) 

# en detección de nombres primero ir a la fija,  luego filtrar los menos comunes para clasificar y los otros casos a manito
# INCLUIR OTROS ATRIBUTOS COMO EL DEPARTAMENTO 
# PRIMER NOMBRE .. ES EL DETERMINANTE DEL GÉNERO

df_clean2.count()

In [ ]:
df_dir_common = df_clean2.withColumn('word', F.explode(F.split(F.col('DIRECCION_CLEAN'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

df_dir_common.show()

In [ ]:
df_dir_common.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/sin_info_dir_palabras_comunes3.json")

In [ ]:
#no es necesario---
#df_clean3 = df_clean2.filter( (df_clean2.DIRECCION_CLEAN.contains("CAMPESTRE") == True )  |  (df_clean2.DIRECCION_CLEAN.contains("PARCELA") == True ) )
#df_dir_common = df_clean3.withColumn('word', F.explode(F.split(F.col('DIRECCION_CLEAN'), ' ')))\
#    .groupBy('word')\
#    .count()\
#    .sort('count', ascending=False)
#df_dir_common.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/sin_info_dir_palabras_comunes_rurales_3.json")
#df_clean3.count()

In [ ]:
#df_clean3.display()

In [ ]:
df1.createOrReplaceTempView("transacciones")
spark.sql("select * from transacciones " ).show()

In [ ]:
df2_sin_duplicados = df2.dropDuplicates()
df2_sin_duplicados.count()